In [1]:
%cd ~/ipython-notebook/dqn4wirelesscontrol/
import sys
import time
import pandas as pd
import lasagne
pd.set_option('mode.chained_assignment', None)  # block warnings due to DataFrame value assignment
# Project modules
from sleep_control.integration import Emulation
import sleep_control.traffic_emulator
import sleep_control.traffic_server
from sleep_control.controller import QController, DummyController
import qlearning.qtable
import qlearning.qnn
from qlearning.qtable import QAgent
from qlearning.qnn import QAgentNN
from qlearning.mixin import PhiMixin
std_out = sys.stdout
class QAgentNNPhi(PhiMixin, QAgentNN):
    def __init__(self, **kwargs):
        super(QAgentNNPhi, self).__init__(**kwargs)

Using gpu device 0: GeForce GTX TITAN X (CNMeM is disabled, CuDNN 4007)
/usr/local/lib/python2.7/dist-packages/theano/tensor/signal/downsample.py:5: UserWarning: downsample module has been moved to the pool module.
  warnings.warn("downsample module has been moved to the pool module.")


/home/admin-326/Data/ipython-notebook/dqn4wirelesscontrol


In [2]:
fake_df = pd.DataFrame()
head_dt = pd.to_datetime("2014-9-15 09:40:00")
tail_dt = pd.to_datetime("2015-01-31 15:36:36") 

---

In [9]:
gamma, alpha = 0.5, 0.9
explore_strategy, epsilon = 'epsilon', 0.02
reward_scaling, reward_scaling_update, rs_period = 1, 'adaptive', 32
phi_length = 5
range_state_slice = [(0, 10), (0, 10), (0, 10), (0, 1), (0, 1)]
momentum, learning_rate, update_period = 0.9, 0.01, 16
num_buffer, memory_size, batch_size, freeze_period  = 2, 200, 100, 4
log_file = "sleep_control/experiments/log/message_2016-6-17_Poisson1_G5_BUF2_AR1_1.log"
time_step, n_days = pd.Timedelta(seconds=2), 7
TOTAL_EPOCHS = 3600*24*n_days/2
backoff = num_buffer*memory_size+phi_length
head_datetime = pd.to_datetime("2014-9-15 09:40:00") - time_step*backoff
beta=0.5
mu = 1

In [10]:
log_file = open(log_file,"w")
dim_state = (1, phi_length, 3+2)
actions = [(True, None), (False, 'serve_all')]
tail_datetime = head_datetime + pd.Timedelta(days=1)*n_days*5
te = sleep_control.traffic_emulator.PoissonEmulator(session_df=fake_df, head_datetime=head_dt, tail_datetime=tail_dt, time_step=time_step, 
                                                    mu=mu, verbose=2)
ts = sleep_control.traffic_server.TrafficServer(cost=5, verbose=2)
agent = QAgentNNPhi(phi_length=phi_length, dim_state=dim_state, range_state=[[range_state_slice]*phi_length], actions=actions,
        alpha=alpha, learning_rate=learning_rate, momentum=momentum, update_period=update_period,
        reward_scaling=reward_scaling, reward_scaling_update=reward_scaling_update, rs_period=rs_period,
        memory_size=memory_size, num_buffer=num_buffer, freeze_period=freeze_period, batch_size=batch_size,
        gamma=gamma, explore_strategy=explore_strategy, epsilon=epsilon,
        verbose=2)
c = QController(agent=agent)
emu = sleep_control.integration.Emulation(te=te, ts=ts, c=c, beta=beta)
t = time.time()
sys.stdout = log_file
while emu.epoch is not None and emu.epoch<TOTAL_EPOCHS:
    # log time
    print "Epoch {}, ".format(emu.epoch),
    left = emu.te.head_datetime + emu.te.epoch*emu.te.time_step
    right = left + emu.te.time_step
    print "{} - {}".format(left.strftime("%Y-%m-%d %H:%M:%S"), right.strftime("%Y-%m-%d %H:%M:%S"))
    emu.step()
    print
sys.stdout = std_out
log_file.close()
time.time()-t

New TrafficEmulator with parameters:
  head=2014-09-15 09:40:00
  tail=2015-01-31 15:36:36
  time_step=0 days 00:00:02
  epoch=0
  verbose=2


1715.228019952774

---

In [11]:
gamma, alpha = 0.5, 0.9
explore_strategy, epsilon = 'epsilon', 0.02
reward_scaling, reward_scaling_update, rs_period = 1, 'adaptive', 32
phi_length = 5
range_state_slice = [(0, 10), (0, 10), (0, 10), (0, 1), (0, 1)]
momentum, learning_rate, update_period = 0.9, 0.01, 16
num_buffer, memory_size, batch_size, freeze_period  = 2, 200, 100, 4
log_file = "sleep_control/experiments/log/message_2016-6-17_Poisson1_G5_BUF2_AR1_2.log"
time_step, n_days = pd.Timedelta(seconds=2), 7
TOTAL_EPOCHS = 3600*24*n_days/2
backoff = num_buffer*memory_size+phi_length
head_datetime = pd.to_datetime("2014-9-15 09:40:00") - time_step*backoff
beta=0.5
mu = 1

In [12]:
log_file = open(log_file,"w")
dim_state = (1, phi_length, 3+2)
actions = [(True, None), (False, 'serve_all')]
tail_datetime = head_datetime + pd.Timedelta(days=1)*n_days*5
te = sleep_control.traffic_emulator.PoissonEmulator(session_df=fake_df, head_datetime=head_dt, tail_datetime=tail_dt, time_step=time_step, 
                                                    mu=mu, verbose=2)
ts = sleep_control.traffic_server.TrafficServer(cost=5, verbose=2)
agent = QAgentNNPhi(phi_length=phi_length, dim_state=dim_state, range_state=[[range_state_slice]*phi_length], actions=actions,
        alpha=alpha, learning_rate=learning_rate, momentum=momentum, update_period=update_period,
        reward_scaling=reward_scaling, reward_scaling_update=reward_scaling_update, rs_period=rs_period,
        memory_size=memory_size, num_buffer=num_buffer, freeze_period=freeze_period, batch_size=batch_size,
        gamma=gamma, explore_strategy=explore_strategy, epsilon=epsilon,
        verbose=2)
c = QController(agent=agent)
emu = sleep_control.integration.Emulation(te=te, ts=ts, c=c, beta=beta)
t = time.time()
sys.stdout = log_file
while emu.epoch is not None and emu.epoch<TOTAL_EPOCHS:
    # log time
    print "Epoch {}, ".format(emu.epoch),
    left = emu.te.head_datetime + emu.te.epoch*emu.te.time_step
    right = left + emu.te.time_step
    print "{} - {}".format(left.strftime("%Y-%m-%d %H:%M:%S"), right.strftime("%Y-%m-%d %H:%M:%S"))
    emu.step()
    print
sys.stdout = std_out
log_file.close()
time.time()-t

New TrafficEmulator with parameters:
  head=2014-09-15 09:40:00
  tail=2015-01-31 15:36:36
  time_step=0 days 00:00:02
  epoch=0
  verbose=2


1721.2680809497833

---

In [13]:
gamma, alpha = 0.5, 0.9
explore_strategy, epsilon = 'epsilon', 0.02
reward_scaling, reward_scaling_update, rs_period = 1, 'adaptive', 32
phi_length = 5
range_state_slice = [(0, 10), (0, 10), (0, 10), (0, 1), (0, 1)]
momentum, learning_rate, update_period = 0.9, 0.01, 16
num_buffer, memory_size, batch_size, freeze_period  = 2, 200, 100, 4
log_file = "sleep_control/experiments/log/message_2016-6-17_Poisson1_G5_BUF2_AR1_3.log"
time_step, n_days = pd.Timedelta(seconds=2), 7
TOTAL_EPOCHS = 3600*24*n_days/2
backoff = num_buffer*memory_size+phi_length
head_datetime = pd.to_datetime("2014-9-15 09:40:00") - time_step*backoff
beta=0.5
mu = 1

In [14]:
log_file = open(log_file,"w")
dim_state = (1, phi_length, 3+2)
actions = [(True, None), (False, 'serve_all')]
tail_datetime = head_datetime + pd.Timedelta(days=1)*n_days*5
te = sleep_control.traffic_emulator.PoissonEmulator(session_df=fake_df, head_datetime=head_dt, tail_datetime=tail_dt, time_step=time_step, 
                                                    mu=mu, verbose=2)
ts = sleep_control.traffic_server.TrafficServer(cost=5, verbose=2)
agent = QAgentNNPhi(phi_length=phi_length, dim_state=dim_state, range_state=[[range_state_slice]*phi_length], actions=actions,
        alpha=alpha, learning_rate=learning_rate, momentum=momentum, update_period=update_period,
        reward_scaling=reward_scaling, reward_scaling_update=reward_scaling_update, rs_period=rs_period,
        memory_size=memory_size, num_buffer=num_buffer, freeze_period=freeze_period, batch_size=batch_size,
        gamma=gamma, explore_strategy=explore_strategy, epsilon=epsilon,
        verbose=2)
c = QController(agent=agent)
emu = sleep_control.integration.Emulation(te=te, ts=ts, c=c, beta=beta)
t = time.time()
sys.stdout = log_file
while emu.epoch is not None and emu.epoch<TOTAL_EPOCHS:
    # log time
    print "Epoch {}, ".format(emu.epoch),
    left = emu.te.head_datetime + emu.te.epoch*emu.te.time_step
    right = left + emu.te.time_step
    print "{} - {}".format(left.strftime("%Y-%m-%d %H:%M:%S"), right.strftime("%Y-%m-%d %H:%M:%S"))
    emu.step()
    print
sys.stdout = std_out
log_file.close()
time.time()-t

New TrafficEmulator with parameters:
  head=2014-09-15 09:40:00
  tail=2015-01-31 15:36:36
  time_step=0 days 00:00:02
  epoch=0
  verbose=2


1718.028095960617